# Heart Attack Detection using CNN 

### Importing the required Libraries

In [2]:
import os
import sys
from time import time
import tensorflow
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.keras.layers import  Convolution2D, MaxPooling2D, ZeroPadding2D

### Dataset path

In [3]:
INPUT_PATH_TRAIN = "B:/Vedant/Academic/Heart-Attack-Detection-In-Images-master/Heart-Attack-Detection-In-Images-master/dataset/train"
INPUT_PATH_VAL = "B:/Vedant/Academic/Heart-Attack-Detection-In-Images-master/Heart-Attack-Detection-In-Images-master/dataset/val"
INPUT_PATH_TEST = "B:/Vedant/Academic/Heart-Attack-Detection-In-Images-master/Heart-Attack-Detection-In-Images-master/dataset/test"
OUTPUT_DIR = "B:/Vedant/Academic/Heart-Attack-Detection-In-Images-master/Heart-Attack-Detection-In-Images-master/model"

In [4]:
# Checkpoints
EPOCH_CHECK_POINT = 2   # How many epoch til save next checkpoint
NUM_CHECK_POINT = 10    # How many epoch will be saved
KEEP_ONLY_LATEST = False# Keeping only the last checkpoint

In [10]:
# Train configurations
WIDTH, HEIGHT = 256, 256# Size images to train
STEPS = 450             # How many steps per epoch
VALIDATION_STEPS = 100  # How many steps per next validation
BATCH_SIZE = 48         # How many images at the same time, change depending on your GPU
LR = 0.003              # Learning rate
CLASSES = 2             # Don't chage, 0=Infarct, 1=Normal

In [6]:
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [7]:
K.clear_session()

train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_gen = train_datagen.flow_from_directory(
    INPUT_PATH_TRAIN,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical')
val_gen = val_datagen.flow_from_directory(
    INPUT_PATH_VAL,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical')
test_gen = test_datagen.flow_from_directory(
    INPUT_PATH_TEST,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 22344 images belonging to 2 classes.
Found 4788 images belonging to 2 classes.
Found 4788 images belonging to 2 classes.


In [8]:
NET = Sequential()
NET.add(Convolution2D(64, kernel_size=(3 ,3), padding ="same", input_shape=(256, 256, 3), activation='relu'))
NET.add(MaxPooling2D((3,3), strides=(3,3)))
NET.add(Convolution2D(128, kernel_size=(3, 3), activation='relu'))
NET.add(MaxPooling2D((3,3), strides=(3,3)))
NET.add(Convolution2D(256, kernel_size=(3, 3), activation='relu'))
NET.add(MaxPooling2D((2,2), strides=(2,2)))
NET.add(Convolution2D(512, kernel_size=(3, 3), activation='relu'))
NET.add(MaxPooling2D((2,2), strides=(2,2)))
NET.add(Convolution2D(1024, kernel_size=(3, 3), activation='relu'))
NET.add(MaxPooling2D((2,2), strides=(2,2)))
NET.add(Dropout(0.3))
NET.add(Flatten())

for _ in range(5):
    NET.add(Dense(128, activation='relu'))
NET.add(Dropout(0.5))

for _ in range(5):
    NET.add(Dense(128, activation='relu'))
NET.add(Dropout(0.5))

for _ in range(5):
    NET.add(Dense(128, activation='relu'))
NET.add(Dropout(0.5))

NET.add(Dense(CLASSES, activation='softmax'))

sgd = SGD(lr=LR, decay=1e-4, momentum=0.9, nesterov=True)

NET.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['acc', 'mse'])

NET.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 85, 85, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 83, 83, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 512)       1

C:\Users\Sushant Kamble\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [ ]:
for i in range(NUM_CHECK_POINT):        
    NET.fit_generator(
        train_gen,
        steps_per_epoch=STEPS,
        epochs=EPOCH_CHECK_POINT,
        validation_data=val_gen,
        validation_steps=VALIDATION_STEPS,
        verbose=1
    )   
    
    print('Saving model: {:02}.'.format(i))
    NET.save(OUTPUT_DIR + "{:02}_model1.h5".format(i))

Epoch 1/2
450/450 [==============================] - 1864s 4s/step - loss: 0.4171 - acc: 0.8272 - mse: 0.1327 - val_loss: 0.5813 - val_acc: 0.7306 - val_mse: 0.2225
Epoch 2/2
450/450 [==============================] - 1808s 4s/step - loss: 0.3367 - acc: 0.8658 - mse: 0.1052 - val_loss: 0.4975 - val_acc: 0.7891 - val_mse: 0.1704
Saving model: 00.
Epoch 1/2
450/450 [==============================] - 1790s 4s/step - loss: 0.2815 - acc: 0.8877 - mse: 0.0873 - val_loss: 0.6070 - val_acc: 0.7389 - val_mse: 0.2138
Epoch 2/2
450/450 [==============================] - 1757s 4s/step - loss: 0.2365 - acc: 0.9069 - mse: 0.0716 - val_loss: 0.5100 - val_acc: 0.8024 - val_mse: 0.1654
Saving model: 01.
Epoch 1/2
450/450 [==============================] - 2066s 5s/step - loss: 0.1836 - acc: 0.9297 - mse: 0.0546 - val_loss: 0.5940 - val_acc: 0.7865 - val_mse: 0.1835
Epoch 2/2
450/450 [==============================] - 1824s 4s/step - loss: 0.1447 - acc: 0.9461 - mse: 0.0426 - val_loss: 0.6936 - val_acc: